In [1]:
import random
import time
import math

from pathlib import Path
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as ipw
from ipywidgets import *

from IPython.display import clear_output, display, Markdown

In [2]:
#plot the charts right into your Jupyter Notebook.
%matplotlib inline 

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
   return false;
}

<IPython.core.display.Javascript object>

In [4]:
out_fetch = ipw.Output(layout=Layout(width = '1000px', border='solid'))
out_read = ipw.Output(layout=Layout(width = '1000px', border='solid'))
out_plot = ipw.Output(layout=Layout(width = '1000px', border='solid'))
out_average = ipw.Output(layout=Layout(width = '1000px', border='solid'))

localFileName = "__lammps_file__"
workingFile = ["None"]
listOfFiles = []
listOfFilesLocal = []

def fetch(btn):
    with out_fetch:
        out_fetch.clear_output()

    with out_read:
        out_read.clear_output()

    with out_plot:
        out_plot.clear_output()

    with out_average:
        out_average.clear_output()

    n = 0
    for file in fileName.value.split():
        flocal = localFileName + str(n)
        with out_fetch:
            print(file)
            command = "scp " + file + " " + flocal
#             error = os.system(command)
#         if error != 0:
#          print("Failed fetching : ",file)
        listOfFiles.append(file)
        listOfFilesLocal.append(flocal)
        n = n + 1

    createMenu(listOfFiles)
    
def read(btn):
    global data
    with out_read:
        out_read.clear_output()

    with out_plot:
        out_plot.clear_output()

    with out_average:
        out_average.clear_output()

    data = readDataLammps(workingFile[0])
    
def reset(btn):
    with out_fetch:
        out_fetch.clear_output()

    with out_read:
        out_read.clear_output()

    with out_average:
        out_average.clear_output()
        
    with out_plot:
        out_plot.clear_output()

    for f in listOfFilesLocal:
        if os.path.exists(f):
            os.remove(f)
        
    try:
        del data
    except:
        pass

In [5]:
def dropdown_eventhandler(change):
    for i in range(0,len(listOfFiles)):
        if listOfFiles[i] == change.new:
            workingFile[0] = listOfFilesLocal[i]

def createMenu(data):
    if len(data) == 1:
        option_list = data
        workingFile[0] = localFileName+"0"
    else:
        option_list = ["Choose one"]
        option_list.extend(data)

    dropdown = Dropdown(description="Active file:", options=option_list, layout={'width':'800px'})
    dropdown.observe(dropdown_eventhandler, names='value')
    with out_fetch:
        display(dropdown)
        

In [6]:
def readDataLammps(localFILE):
    global fields
    fields = []

    nData = 0
    nSkip = 100

    iBlock = 0
    lines = []
    with open(localFILE, 'r') as file:
        for line in file:
            line = line.strip() # removes spaces

            if "Step" in line:
                iBlock = abs(iBlock) + 1
                header = line.split()
                continue
            elif "Loop" in line:
                iBlock = -iBlock

            elif "WARNING" in line:
                continue

            if iBlock > 0:
                values = line.split()
                nData = nData + 1
                lines.append(values)

        data = pd.DataFrame(lines)

    with out_read:
        print("Number of data points read    :",len(data))
        print("------------------------------")

        nf = 15
        string = ''
        for i in range(0,len(header)):
            string = string.strip() + " " +header[i]
            if (i+1) % nf == 0: 
                print(string)
                string = ''
        if len(header) != nf:
            print(string)
        print("------------------------------")
        
    data.columns = header

    data = data.apply(pd.to_numeric)

    for f in header:
        if f in ("Step", "Time", "CPU", "CPULeft"):
            continue
        fields.append(f)
    
    return data

In [7]:
def average(btn):
    with out_average:
        out_average.clear_output()

    x = data["Time"]
    n = len(x)
    t0 = float(avgTime0.value) / 100
    t1 = float(avgTime1.value) / 100

    i0 = max(int(n*t0)-1,0)
    i1 = min(int(n*t1)-1,n)

    with out_average:
        print("Start time for average        :" , x[i0])
        print("Final time for average        :" , x[i1])
        print("------------------------------")

    for f in fields:
        if f in ("Step", "Time", "CPU", "CPULeft"):
            continue
        
        if (f in fieldPlot.value) or fieldPlot.value == "All" :
            y = data[f]

            avg = np.mean(y[i0:i1])
            stdev = np.std(y[i0:i1])
            stder = np.std(y[i0:i1]) / np.sqrt(i1-i0+1)

            with out_average:
                print("Field                         :" , f)
                print("Average - StdErr - Stdev      :" , avg, stder, stdev)
                print("------------------------------")

In [8]:
def makePlot(btn):
    with out_plot:
        out_plot.clear_output()

    x = data["Time"]
    n = len(x)
    t0 = float(plotTime0.value) / 100
    t1 = float(plotTime1.value) / 100

    i0 = max(int(n*t0)-1,0)
    i1 = min(int(n*t1)-1,n)
  
    with out_plot:
        print("Start time for plotting       :" , x[i0])
        print("Final time for plotting       :" , x[i1])
        print("------------------------------")
        print(" ")

    for f in fields:
        if (f in fieldPlot.value) or fieldPlot.value == "All" :            
            y = data[f]

            fig = plt.figure(figsize=(30, 5))
            ax = fig.gca()

            fs = 24
            ax.plot(x[i0:i1],y[i0:i1])
            plt.xlabel("Time [ps]", fontsize=fs)
            plt.xticks(fontsize=fs)
            plt.ylabel(f, fontsize=fs)
            plt.yticks(fontsize=fs)

            fit = np.polyfit(x[i0:i1],y[i0:i1],1)
            drift = np.poly1d(fit)
            xdrift = range(int(x[i0]),int(x[i1]))
            ydrift = drift(xdrift)
            ax.plot(xdrift,ydrift,c='red')

            lw = 3
            ax.spines["top"].set_linewidth(lw)
            ax.spines["bottom"].set_linewidth(lw)
            ax.spines["left"].set_linewidth(lw)
            ax.spines["right"].set_linewidth(lw)

            with out_plot:
                plt.show()

In [9]:
# --- Interactive actions ---
actions = []

# --- Input files ---
fileName = Textarea(value='',
                    placeholder='Enter LAMMPS outputfile location here',
                    description='',
                    rows=3,                  
                    layout=Layout(width="auto"))
actions.append(fileName)

# --- Fetch files button ---
btn_fetch = ipw.Button(description="Fetch", layout=ipw.Layout(width="150px"))
btn_fetch.on_click(fetch)

# --- Clear all button ---
btn_reset = ipw.Button(description="Clear", layout=ipw.Layout(width="150px"))
btn_reset.on_click(reset)
reset(btn_reset)

# --- Display buttons ---
actions.append(ipw.HBox([btn_fetch,btn_reset]))
actions.append(ipw.HBox([out_fetch]))

# --- Read button chosen from dropdown menu ---
btn_read = ipw.Button(description="Read", layout=ipw.Layout(width="150px"))
btn_read.on_click(read)
actions.append(ipw.HBox([btn_read]))
actions.append(ipw.HBox([out_read]))

########################################################
# Plot section : sliders and button
plotTime0 = ipw.FloatSlider(value=0, min=0, max=100)
plotTime1 = ipw.FloatSlider(value=100, min=0, max=100)
s1 = ipw.HBox([ipw.Label(value="Graph start time (%)"), plotTime0])
s2 = ipw.HBox([ipw.Label(value="Graph final time (%)"), plotTime1])
s3 = ipw.HBox([s1,s2])
actions.append(s3)

btn_plot = ipw.Button(description="Make graph for", layout=ipw.Layout(width="150px"))
btn_plot.on_click(makePlot)
fieldPlot = ipw.Text("All", layout=ipw.Layout(width="850px"))
fieldPlotBox = ipw.HBox([fieldPlot])
actions.append(ipw.HBox([btn_plot,fieldPlotBox]))
actions.append(ipw.HBox([out_plot]))

########################################################
# Average section : : sliders and button
avgTime0 = ipw.FloatSlider(value=0, min=0, max=100)
avgTime1 = ipw.FloatSlider(value=100, min=0, max=100)
s4 = (ipw.HBox([ipw.Label(value="Average start time (%)"), avgTime0]))
s5 = (ipw.HBox([ipw.Label(value="Average final time (%)"), avgTime1]))
s6 = ipw.HBox([s4,s5])
actions.append(s6)

btn_average = ipw.Button(description="Compute Average for", layout=ipw.Layout(width="150px"))
btn_average.on_click(average)
fieldAverage = ipw.Text("All", layout=ipw.Layout(width="850px"))
fieldAverageBox = ipw.HBox([fieldAverage])
actions.append(ipw.HBox([btn_average,fieldAverageBox]))
actions.append(ipw.HBox([out_average]))

###########################
ipw.VBox(actions)
